In [ ]:
import tflearn
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.estimator import regression

In [ ]:
import pickle
import numpy as np
from random import shuffle
import os
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

In [ ]:
with open('lexicon.pickle','rb') as f:
    lexicon = pickle.load(f)

In [ ]:
with open('train_test_data.pickle','rb') as f:
    train_x, test_x, Y, test_y = pickle.load(f)

In [ ]:
X = np.array(train_x).reshape(-1, 1, len(lexicon), 1)
test_x = np.array(test_x).reshape(-1, 1, len(lexicon), 1)

In [ ]:
LR = 1e-3
MODEL_NAME = 'sentiment-{}-{}.model'.format(LR, '6conv-basic')

In [ ]:
convnet = input_data(shape=[None, 1, len(lexicon), 1], name='input')

convnet = conv_2d(convnet, 32, 2, activation='relu')
convnet = max_pool_2d(convnet, 2)

convnet = conv_2d(convnet, 64, 2, activation='relu')
convnet = max_pool_2d(convnet, 2)

convnet = conv_2d(convnet, 32, 2, activation='relu')
convnet = max_pool_2d(convnet, 2)

convnet = conv_2d(convnet, 64, 2, activation='relu')
convnet = max_pool_2d(convnet, 2)

convnet = conv_2d(convnet, 32, 2, activation='relu')
convnet = max_pool_2d(convnet, 2)

convnet = conv_2d(convnet, 64, 2, activation='relu')
convnet = max_pool_2d(convnet, 2)

convnet = fully_connected(convnet, 1024, activation='relu')
convnet = dropout(convnet, 0.8)

convnet = fully_connected(convnet, 2, activation='softmax')
convnet = regression(convnet, optimizer='adam', learning_rate=LR, loss='categorical_crossentropy', name='targets')

model = tflearn.DNN(convnet, tensorboard_dir='log')

In [ ]:
if os.path.exists("{}.meta".format(MODEL_NAME)):
    model.load(MODEL_NAME)
    print('Model loaded!')

In [ ]:
#model.fit({'input': X}, {'targets': Y}, n_epoch=10, validation_set=({'input': test_x}, {'targets': test_y}),snapshot_step=500, show_metric=True, run_id=MODEL_NAME)
#model.save(MODEL_NAME)

In [ ]:
def process_prediction_sample(sentence):
    words = word_tokenize(sentence)
    words = [lemmatizer.lemmatize(i) for i in words]
    
    features = np.zeros(len(lexicon))
    for word in words:
        if word.lower() in lexicon:
            index_value = lexicon.index(word.lower())
            features[index_value] = 1
    
    return features

In [36]:
input_data = "You're awesome! So am I"

features = process_prediction_sample(input_data)
features = np.array(features).reshape(-1, 1, len(lexicon), 1)

model_out = model.predict(features)[0]
if np.argmax(model_out) == 0:
    str_label = 'ham'
else:
    str_label = 'spam'

print("This message is a \"",str_label,"\"")

This message is a " ham "
